In [1]:
!pip install gym[classic_control]
!pip install keras-rl2
!pip install protobuf==3.20.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 909.1 kB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras as k
import numpy as np
import gym
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from keras import __version__
tf.keras.__version__ = __version__

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


from gym import Env
from gym.spaces import Discrete, Box, MultiBinary
import numpy as np
import random

from collections import deque

# 1. Test Random Environment with OpenAI Gym

In [3]:
def tit_for_tat(round,opp_move):
  if round == 200:
    return 1
  else:
    return opp_move

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
def cooperate():
  return 1

def defect():
  return 0

def rand():
  return random.randint(0,1)

In [5]:
def tit_for_two_tat(round, opp_move1, opp_move2):
  if round >= 199 or opp_move1 == 1 or opp_move2 == 1:
    return 1
  else:
    return 0

In [6]:
def grudger(defected):
  if defected:
    return 0
  else:
    return 1

In [7]:
def pavlov(round, last_move,opp_move):
  if round == 200:
    return 1
  elif last_move == opp_move or (last_move == 0 and opp_move == 1):
    return last_move
  elif last_move == 1 and opp_move == 0:
    return 0

In [8]:
def soft_grudger(last_defect):
  if last_defect < 4:
    return 0
  else:
    return 1

In [9]:
def fortress(round, opp_move1, opp_move2):
  if round == 200 or opp_move1 == 0 or opp_move2 == 0:
    return 0
  else:
    return 1

In [10]:
def alternator(round,last_move):
  if round == 200 or last_move == 0:
    return 1
  else:
    return 0

In [11]:
def sneaky(round,opp_move):
  dice=random.randint(0,9)
  if round == 200:
    return 1
  elif dice == 0:
    return 0;
  else:
    return opp_move

In [12]:
def custom(round):
  if round%4 == 0:
    return 0
  else:
    return 1

# last 10 moves environment

In [13]:
class PrisonEnv(Env):
    def __init__(self):
        self.action_space = Discrete(2)
        self.observation_space = MultiBinary(10)
        self.round = 200
        self.state = np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
        self.opp_strategy = random.randint(0,10)
        self.opp_move1=1
        self.opp_move2=1
        self.last_move1=self.state[0]
        self.last_move2=self.state[1]
        self.opp_defected=False
        self.user_defected=False
        self.last_defect= np.Infinity
        self.opp_last_defect= np.Infinity
        self.move=1
        self.opp_move=1

        self.opp_score=0

    def step(self, action):

        self.opp_move2=self.opp_move1
        self.opp_move1=self.opp_move
        self.last_move2=self.state[1]
        self.last_move1=self.state[0]

        self.move=action



        if self.opp_strategy == 0:
          opp_move=tit_for_tat(self.round,self.last_move1)
        elif self.opp_strategy == 1:
          opp_move=cooperate()
        elif self.opp_strategy == 2:
          opp_move=sneaky(self.round, self.last_move1)
        elif self.opp_strategy == 3:
          opp_move=rand()
        elif self.opp_strategy == 4:
          opp_move=tit_for_two_tat(self.round,self.last_move1, self.last_move2)
        elif self.opp_strategy == 5:
          opp_move=grudger(self.user_defected)
        elif self.opp_strategy == 6:
          opp_move=pavlov(self.round,self.opp_move1, self.last_move1)
        elif self.opp_strategy == 7:
          opp_move=soft_grudger(self.last_defect)
        elif self.opp_strategy == 8:
          opp_move=fortress(self.round, self.last_move1, self.last_move2)
        elif self.opp_strategy == 9:
          opp_move=alternator(self.round,self.opp_move1)
        elif self.opp_strategy == 10:
          opp_move = custom(self.round)






        temp = np.array(list(self.state))

        for i in range(1,10):
          self.state[i] = temp[i-1]
        self.state[0] = self.move


        if opp_move == 0:
          self.opp_defected=True
          self.opp_last_defect=0
        else:
          self.opp_last_defect += 1

        if self.move == 0:
          self.user_defected=True
          self.last_defect=0
        else:
          self.last_defect += 1

        if self.move == 1 and opp_move == 1:
          reward = 3
          self.opp_score += 3
        elif self.move == 1 and opp_move == 0:
          reward = 0
          self.opp_score += 5
        elif self.move == 0 and opp_move == 1:
          reward = 5
          self.opp_score += 0
        elif self.move == 0 and opp_move == 0:
          reward = 1
          self.opp_score += 1

        self.round -= 1

        if self.round <= 0:
            done = True

        else:
            done = False

        info = {'opp_score':self.opp_score}



        return self.state, reward, done, info


    def reset(self):
        self.round = 200
        self.state = np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
        self.opp_strategy = random.randint(0,10)
        self.opp_move1=1
        self.opp_move2=1
        self.last_move1=self.state[0]
        self.last_move2=self.state[1]
        self.opp_defected=False
        self.user_defected=False
        self.last_defect= np.Infinity
        self.opp_last_defect= np.Infinity
        self.move=1
        self.opp_move=1

        self.opp_score = 0

        return self.state

In [14]:
env = PrisonEnv()
env.state

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [15]:
data = env.state.reshape(-1, 10)
data[0]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [16]:
env = PrisonEnv()
episodes = 100
done = False
score = 0
wins=0
for episode in range(1,episodes+1):
  state=env.reset()
  done=False
  score=0
  while not done:
      action = env.action_space.sample()
      n_state, reward, done, info = env.step(action)
      score+=reward
  if info['opp_score'] < score:
    wins += 1

  print('Episode:{} Score:{} Info:{}'.format(episode, score, info))

print('Win Rate:',wins/100)


Episode:1 Score:437 Info:{'opp_score': 437}
Episode:2 Score:618 Info:{'opp_score': 388}
Episode:3 Score:271 Info:{'opp_score': 546}
Episode:4 Score:806 Info:{'opp_score': 291}
Episode:5 Score:121 Info:{'opp_score': 611}
Episode:6 Score:106 Info:{'opp_score': 586}
Episode:7 Score:92 Info:{'opp_score': 642}
Episode:8 Score:630 Info:{'opp_score': 355}
Episode:9 Score:149 Info:{'opp_score': 574}
Episode:10 Score:285 Info:{'opp_score': 525}
Episode:11 Score:120 Info:{'opp_score': 535}
Episode:12 Score:420 Info:{'opp_score': 450}
Episode:13 Score:606 Info:{'opp_score': 406}
Episode:14 Score:146 Info:{'opp_score': 591}
Episode:15 Score:646 Info:{'opp_score': 371}
Episode:16 Score:397 Info:{'opp_score': 497}
Episode:17 Score:446 Info:{'opp_score': 446}
Episode:18 Score:437 Info:{'opp_score': 462}
Episode:19 Score:441 Info:{'opp_score': 441}
Episode:20 Score:445 Info:{'opp_score': 440}
Episode:21 Score:99 Info:{'opp_score': 619}
Episode:22 Score:417 Info:{'opp_score': 412}
Episode:23 Score:450 

In [17]:
env.state

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

In [18]:
states = env.observation_space.shape[0]
actions = env.action_space.n

In [19]:
states

10

In [20]:
actions

2

In [21]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(1,states)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(actions, activation='softmax'))
    model.add(Flatten())
    return model

In [23]:
del model

NameError: name 'model' is not defined

In [24]:
model = build_model(states, actions)

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 128)            1408      
                                                                 
 dense_1 (Dense)             (None, 1, 128)            16512     
                                                                 
 dense_2 (Dense)             (None, 1, 128)            16512     
                                                                 
 dense_3 (Dense)             (None, 1, 2)              258       
                                                                 
 flatten (Flatten)           (None, 2)                 0         
                                                                 
Total params: 34690 (135.51 KB)
Trainable params: 34690 (135.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
model.input_shape

(None, 1, 10)

In [27]:
env.state.shape

(10,)

In [28]:
env.state

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

In [29]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [30]:
dqn = build_agent(model, actions)
dqn.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 15:53 - reward: 3.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.10/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   30/10000 [..............................] - ETA: 4:57 - reward: 0.8333

/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch

10000/10000 [==============================] - 90s 9ms/step - reward: 2.0185
50 episodes - episode_reward: 403.700 [141.000, 908.000] - loss: 3.376 - mae: 1.132 - mean_q: 0.999 - opp_score: 165.286

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: 1.8739
50 episodes - episode_reward: 374.780 [143.000, 898.000] - loss: 4.152 - mae: 1.266 - mean_q: 1.000 - opp_score: 169.387

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: 1.6375
50 episodes - episode_reward: 327.500 [142.000, 898.000] - loss: 3.708 - mae: 1.187 - mean_q: 1.000 - opp_score: 178.236

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 92s 9ms/step - reward: 1.9818
50 episodes - episode_reward: 396.360 [138.000, 902.000] - loss: 3.748 - mae: 1.196 - mean_q: 1.000 - opp_score: 166.320

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 94s 9ms/step - re

In [31]:
scores = dqn.test(env, nb_episodes=100, visualize=False,verbose=0)

In [32]:
import numpy as np

wins=0

for episode in range(100):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = dqn.forward(state)
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        state = next_state

        if done:
          if total_reward > info['opp_score']:
              wins +=1


print(f'Win Rate: {wins/100}')


Win Rate: 0.96


In [33]:
class ReplayBuffer:
    def __init__(self, buffer_size=int(1e5)):
        self.buffer = deque(maxlen=buffer_size)

    def store(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)


In [34]:
class MyDQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = ReplayBuffer()
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mae', optimizer=tf.keras.optimizers.legacy.Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def learn(self, batch_size):
        minibatch = self.memory.sample(batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [35]:
env = PrisonEnv()
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = MyDQNAgent(state_size, action_size)
episodes = 20

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    episode_reward=0
    for time in range(200):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        episode_reward += episode_reward
        next_state = np.reshape(next_state, [1, state_size])
        agent.memory.store(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"Episode: {e+1}/{episodes}, Score: {episode_reward}, Epsilon: {agent.epsilon:.2}")
            break
        if len(agent.memory) > 32:
            agent.learn(32)


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Episode: 1/20, Score: 0, Epsilon: 0.43
Episode: 2/20, Score: 0, Epsilon: 0.16
Episode: 3/20, Score: 0, Epsilon: 0.059
Episode: 4/20, Score: 0, Epsilon: 0.022
Episode: 5/20, Score: 0, Epsilon: 0.01
Episode: 6/20, Score: 0, Epsilon: 0.01
Episode: 7/20, Score: 0, Epsilon: 0.01
Episode: 8/20, Score: 0, Epsilon: 0.01
Episode: 9/20, Score: 0, Epsilon: 0.01
Episode: 10/20, Score: 0, Epsilon: 0.01
Episode: 11/20, Score: 0, Epsilon: 0.01
Episode: 12/20, Score: 0, Epsilon: 0.01
Episode: 13/20, Score: 0, Epsilon: 0.01
Episode: 14/20, Score: 0, Epsilon: 0.01
Episode: 15/20, Score: 0, Epsilon: 0.01
Episode: 16/20, Score: 0, Epsilon: 0.01
Episode: 17/20, Score: 0, Epsilon: 0.01
Episode: 18/20, Score: 0, Epsilon: 0.01
Episode: 19/20, Score: 0, Epsilon: 0.01
Episode: 20/20, Score: 0, Epsilon: 0.01


In [36]:
agent.epsilon = 0
test_episodes = 100
wins=0

for episode in range(test_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    episode_reward = 0

    for t in range(500):
        action = agent.act(state)
        state, reward, done, info = env.step(action)
        state = np.reshape(state, [1, state_size])
        episode_reward += reward

        if done:
          if episode_reward > info['opp_score']:
            wins += 1
            break

    print(f"Test Episode {episode+1}/{test_episodes} Reward: {episode_reward} Opponent Score: {info['opp_score']}")

print(f"Win Rate: {wins/100}")


Test Episode 1/100 Reward: 592 Opponent Score: 102
Test Episode 2/100 Reward: 204 Opponent Score: 199
Test Episode 3/100 Reward: 208 Opponent Score: 198
Test Episode 4/100 Reward: 208 Opponent Score: 198
Test Episode 5/100 Reward: 1000 Opponent Score: 0
Test Episode 6/100 Reward: 204 Opponent Score: 199
Test Episode 7/100 Reward: 800 Opponent Score: 50
Test Episode 8/100 Reward: 204 Opponent Score: 199
Test Episode 9/100 Reward: 204 Opponent Score: 199
Test Episode 10/100 Reward: 204 Opponent Score: 199
Test Episode 11/100 Reward: 204 Opponent Score: 199
Test Episode 12/100 Reward: 616 Opponent Score: 96
Test Episode 13/100 Reward: 204 Opponent Score: 199
Test Episode 14/100 Reward: 204 Opponent Score: 199
Test Episode 15/100 Reward: 204 Opponent Score: 199
Test Episode 16/100 Reward: 204 Opponent Score: 199
Test Episode 17/100 Reward: 204 Opponent Score: 199
Test Episode 18/100 Reward: 1000 Opponent Score: 0
Test Episode 19/100 Reward: 204 Opponent Score: 199
Test Episode 20/100 Rewar

# Three Players


In [37]:
def tit_for_tat_all(round,opp1_move,opp2_move):
  if round == 200:
    return 1
  else:
    return opp1_move and opp2_move

def tit_for_tat_any(round,opp1_move,opp2_move):
  if round == 200:
    return 1
  else:
    return opp1_move or opp2_move

In [38]:
def cooperate():
  return 1

def defect():
  return 0

def rand():
  return random.randint(0,1)

In [39]:
def tit_for_two_tat_any(round, opp1_move1, opp1_move2, opp2_move1, opp2_move2):
  if round >= 199 or opp1_move1 == 1 or opp1_move1 == 1 or opp2_move1 or opp2_move2:
    return 1
  else:
    return 0

def tit_for_two_tat_all(round, opp1_move1, opp1_move2, opp2_move1, opp2_move2):
  if round >= 199 or (opp1_move1 == 1 or opp1_move2 == 1) and (opp2_move1 or opp2_move2):
    return 1
  else:
    return 0

In [40]:
def soft_grudger_any(last_defect1, last_defect2):
  if last_defect1 < 4 or last_defect2 < 4:
    return 0
  else:
    return 1

def soft_grudger_all(last_defect1, last_defect2):
  if last_defect1 < 4 and last_defect2 < 4:
    return 0
  else:
    return 1

In [41]:
class ThreePlayers():
    def __init__(self):
        self.action_space = Discrete(2)                      # 0:defect 1:cooperate
        self.observation_space = np.array([MultiBinary(10), MultiBinary(10)])
        self.round = 200

        self.state = np.array([[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]])

        self.opp1_strategy = random.randint(0,9)
        self.opp1_move1=self.state[0][0]
        self.opp1_move2=self.state[0][1]

        self.opp2_strategy = random.randint(0,9)
        self.opp2_move1=self.state[1][0]
        self.opp2_move2=self.state[1][1]

        self.last_move1=1
        self.last_move2=1

        self.opp1_defected=False
        self.opp2_defected=False
        self.user_defected=False

        self.last_defect= np.Infinity
        self.opp1_last_defect= np.Infinity
        self.opp2_last_defect= np.Infinity
        self.move=np.array([random.randint(0,1),random.randint(0,1)])

        self.opp1_score = 0
        self.opp2_score = 0


    def step(self, action):
        self.opp1_move1=self.state[0][0]
        self.opp1_move2=self.state[0][1]

        self.opp2_move1=self.state[1][0]
        self.opp2_move2=self.state[1][1]

        self.last_move2=self.last_move1
        self.last_move1=self.move

        self.move=action


        if self.opp1_strategy == 0:
          opp1_move=tit_for_tat_any(self.round,self.last_move1, self.opp2_move1)
        elif self.opp1_strategy == 1:
          opp1_move=tit_for_tat_all(self.round,self.last_move1, self.opp2_move1)
        elif self.opp1_strategy == 2:
          opp1_move=tit_for_two_tat_all(self.round, self.last_move1, self.last_move2, self.opp2_move1, self.opp2_move2)
        elif self.opp1_strategy == 3:
          opp1_move=tit_for_two_tat_any(self.round, self.last_move1, self.last_move2, self.opp2_move1, self.opp2_move2)
        elif self.opp1_strategy == 4:
          opp1_move=soft_grudger_any(self.last_defect, self.opp2_last_defect)
        elif self.opp1_strategy == 5:
          opp1_move=soft_grudger_all(self.last_defect, self.opp2_last_defect)

        if self.opp2_strategy == 0:
          opp2_move=tit_for_tat_any(self.round,self.last_move1, self.opp1_move1)
        elif self.opp2_strategy == 1:
          opp2_move=tit_for_tat_all(self.round,self.last_move1, self.opp1_move1)
        elif self.opp2_strategy == 2:
          opp2_move=tit_for_two_tat_all(self.round, self.last_move1, self.last_move2, self.opp1_move1, self.opp1_move2)
        elif self.opp2_strategy == 3:
          opp2_move=tit_for_two_tat_any(self.round, self.last_move1, self.last_move2, self.opp1_move1, self.opp1_move2)
        elif self.opp2_strategy == 4:
          opp2_move=soft_grudger_any(self.last_defect, self.opp1_last_defect)
        elif self.opp2_strategy == 5:
          opp2_move=soft_grudger_all(self.last_defect, self.opp1_last_defect)



        temp = np.array(list(self.state[0]))
        for i in range(1,10):
          self.state[0][i] = temp[i-1]
        self.state[0][0] = opp1_move

        temp = np.array(list(self.state[1]))
        for i in range(1,10):
          self.state[1][i] = temp[i-1]
        self.state[1][0] = opp2_move



        if opp1_move == 0:
          self.opp1_defected=True
          self.opp1_last_defect=0
        else:
          self.opp1_last_defect += 1

        if opp2_move == 0:
          self.opp2_defected=True
          self.opp2_last_defect=0
        else:
          self.opp2_last_defect += 1

        if self.move == 0:
          self.user_defected=True
          self.last_defect=0
        else:
          self.last_defect += 1



        if self.move == 1 and opp1_move == 1 and opp2_move ==  1:
          reward = 4
          self.opp1_score += 4
          self.opp2_score += 4
        elif self.move == 1 and opp1_move == 1 and opp2_move == 0:
          reward = 0
          self.opp1_score += 0
          self.opp2_score += 10
        elif self.move == 1 and opp1_move == 0 and opp2_move == 1:
          reward = 0
          self.opp1_score += 10
          self.opp2_score += 0
        elif self.move == 1 and opp1_move == 0 and opp2_move == 0:
          reward = 0
          self.opp1_score += 5
          self.opp2_score += 5
        elif self.move == 0 and opp1_move == 1 and opp2_move ==  1:
          reward = 10
          self.opp1_score += 0
          self.opp2_score += 0
        elif self.move == 0 and opp1_move == 1 and opp2_move == 0:
          reward = 5
          self.opp1_score += 0
          self.opp2_score += 5
        elif self.move == 0 and opp1_move == 0 and opp2_move == 1:
          reward = 5
          self.opp1_score += 5
          self.opp2_score += 0
        elif self.move == 0 and opp1_move == 0 and opp2_move == 0:
          reward = 2
          self.opp1_score += 2
          self.opp2_score += 2

        self.round -= 1

        if self.round == 0:
            done = True
        else:
            done = False

        info = {'opp1_score':self.opp1_score, 'opp2_score':self.opp2_score}


        # Return step information
        return self.state, reward, done, info

    def render(self,mode):
        # Implement viz
        pass

    def reset(self):
        self.round = 200
        self.opp1_strategy = random.randint(0,5)
        self.opp2_strategy = random.randint(0,5)
        self.state = np.array([[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]])
        self.opp1_move1=self.state[0][0]
        self.opp1_move2=self.state[0][1]
        self.opp2_move1=self.state[1][0]
        self.opp2_move2=self.state[1][1]
        self.last_move1=1
        self.last_move2=1
        self.opp1_defected=False
        self.opp2_defected=False
        self.user_defected=False
        self.last_defect= np.Infinity
        self.opp1_last_defect= np.Infinity
        self.opp2_last_defect= np.Infinity
        self.gamma=0.9
        self.move=random.randint(0,1)
        self.reward = 0

        self.opp1_score = 0
        self.opp2_score = 0

        return self.state

In [42]:
env = ThreePlayers()

In [43]:
env.state

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [44]:
env = ThreePlayers()
episodes = 100
wins=0
for episode in range(1, episodes+1):
    state = env.reset()
    print(env.opp1_strategy, env.opp2_strategy)
    done = False
    score = 0
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    if score > info['opp1_score'] and score > info['opp2_score']:
      wins += 1

    print('Episode:{} Score:{} Info:{}'.format(episode, score, info))
print(wins/100)

2 2
Episode:1 Score:246 Info:{'opp1_score': 686, 'opp2_score': 686}
0 3
Episode:2 Score:1388 Info:{'opp1_score': 408, 'opp2_score': 408}
3 3
Episode:3 Score:1418 Info:{'opp1_score': 388, 'opp2_score': 388}
0 3
Episode:4 Score:1340 Info:{'opp1_score': 440, 'opp2_score': 440}
4 2
Episode:5 Score:208 Info:{'opp1_score': 708, 'opp2_score': 688}
2 1
Episode:6 Score:285 Info:{'opp1_score': 655, 'opp2_score': 690}
3 1
Episode:7 Score:815 Info:{'opp1_score': 305, 'opp2_score': 900}
0 4
Episode:8 Score:402 Info:{'opp1_score': 352, 'opp2_score': 1112}
2 0
Episode:9 Score:1090 Info:{'opp1_score': 635, 'opp2_score': 445}
0 1
Episode:10 Score:700 Info:{'opp1_score': 415, 'opp2_score': 865}
5 5
Episode:11 Score:1424 Info:{'opp1_score': 384, 'opp2_score': 384}
4 0
Episode:12 Score:385 Info:{'opp1_score': 1030, 'opp2_score': 365}
4 2
Episode:13 Score:231 Info:{'opp1_score': 671, 'opp2_score': 666}
4 1
Episode:14 Score:226 Info:{'opp1_score': 681, 'opp2_score': 681}
5 4
Episode:15 Score:237 Info:{'opp1

In [45]:
env.state.shape

(2, 10)

In [46]:
env.observation_space.shape

(2,)

In [47]:
states = env.state.shape
actions = env.action_space.n

In [48]:
states

(2, 10)

In [49]:
actions

2

In [50]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(1, 2, 10), input_dim=3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='softmax'))
    model.add(Flatten())
    return model

In [51]:
del model

In [52]:
model = build_model(states, actions)

In [53]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 1, 2, 128)         1408      
                                                                 
 dense_8 (Dense)             (None, 1, 2, 128)         16512     
                                                                 
 dense_9 (Dense)             (None, 1, 2, 128)         16512     
                                                                 
 dense_10 (Dense)            (None, 1, 2, 1)           129       
                                                                 
 flatten_1 (Flatten)         (None, 2)                 0         
                                                                 
Total params: 34561 (135.00 KB)
Trainable params: 34561 (135.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
model.input_shape

(None, 1, 2, 10)

In [55]:
env.state.shape

(2, 10)

In [56]:
actions

2

In [57]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions)
    return dqn

In [58]:
dqn = build_agent(model, actions)
dqn.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),metrics=['mae'])
#dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=25000, visualize=False)

Training for 25000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 85s 8ms/step - reward: 3.2532
50 episodes - episode_reward: 650.640 [188.000, 1442.000] - loss: 13.853 - mae: 2.403 - mean_q: 1.000 - opp1_score: 296.133 - opp2_score: 299.551

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 94s 9ms/step - reward: 3.2598
50 episodes - episode_reward: 651.960 [184.000, 1430.000] - loss: 11.192 - mae: 2.115 - mean_q: 1.000 - opp1_score: 306.261 - opp2_score: 295.124

Interval 3 (20000 steps performed)
 4998/10000 [=============>................] - ETA: 47s - reward: 3.8201done, took 225.766 seconds


In [59]:
scores = dqn.test(env, nb_episodes=100, visualize=False,verbose=1)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 414.000, steps: 200
Episode 2: reward: 411.000, steps: 200
Episode 3: reward: 411.000, steps: 200
Episode 4: reward: 414.000, steps: 200
Episode 5: reward: 411.000, steps: 200
Episode 6: reward: 2000.000, steps: 200
Episode 7: reward: 414.000, steps: 200
Episode 8: reward: 411.000, steps: 200
Episode 9: reward: 411.000, steps: 200
Episode 10: reward: 411.000, steps: 200
Episode 11: reward: 411.000, steps: 200
Episode 12: reward: 2000.000, steps: 200
Episode 13: reward: 416.000, steps: 200
Episode 14: reward: 2000.000, steps: 200
Episode 15: reward: 2000.000, steps: 200
Episode 16: reward: 419.000, steps: 200
Episode 17: reward: 408.000, steps: 200
Episode 18: reward: 2000.000, steps: 200
Episode 19: reward: 411.000, steps: 200
Episode 20: reward: 411.000, steps: 200
Episode 21: reward: 414.000, steps: 200
Episode 22: reward: 2000.000, steps: 200
Episode 23: reward: 419.000, steps: 200
Episode 24: reward: 411.000, steps: 200
Episode 25: re

In [60]:
import numpy as np

wins=0

for episode in range(100):  # Run 100 episodes
    state = env.reset()  # Reset environment for new episode
    done = False
    total_reward = 0

    while not done:
        action = dqn.forward(state)  # Get action from DQN agent
        next_state, reward, done, info = env.step(action)  # Apply action to environment
        total_reward += reward  # Accumulate reward
        state = next_state  # Move to next state

        if done:
          if total_reward > info['opp1_score'] and total_reward > info['opp2_score']:
              wins +=1


print(f'Win Rate: {wins/100}')


Win Rate: 1.0
